In [8]:
import json
import numpy as np

In [9]:
def linear(x):
    return x

def derivative_linear(x):
    return np.ones(x.shape)

def relu(x):
    return np.maximum(0, x)

def derivative_relu(x):
    return np.where(x < 0, 0, 1)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def derivative_sigmoid(x):
    return sigmoid(x) * (1 - sigmoid(x))

def softmax(x):
    exp_x = np.exp(x)
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

def derivative_softmax(x, isTarget):
    if isTarget:
        return softmax(x)
    else:
        return -(1-softmax(x))

In [10]:
def delta_output(activation_function, target, output, net):
  if activation_function == "linear":
      result = (target - output) * derivative_linear(net)
  elif activation_function == "relu":
      result = (target - output) * derivative_relu(net)
  elif activation_function == "sigmoid":
      result = (target - output) * derivative_sigmoid(net)
  elif activation_function == "softmax":
      result = - derivative_softmax(net, True)

  return result

def delta_hidden(activation_function, net, delta_output, weights):
  if activation_function == "linear":
      result = np.dot(delta_output, weights.T) * derivative_linear(net)
  elif activation_function == "relu":
      result = np.dot(delta_output, weights.T) * derivative_relu(net)
  elif activation_function == "sigmoid":
      result = np.dot(delta_output, weights.T) * derivative_sigmoid(net)
  elif activation_function == "softmax":
      result = - derivative_softmax(net, False)

  return result

def delta_error(activation_function, target, output, net, weights, x, isOutputLayer):
    if isOutputLayer:
        return - delta_output(activation_function, target, output, net) * x
    else:
        return - delta_hidden(activation_function, net, delta_output, weights) * x

In [11]:
def terminate_condition(stopped_by, max_iteration, error_threshold, iteration, error):
    if stopped_by == "max_iteration":
        return iteration >= max_iteration
    elif stopped_by == "error_threshold":
        return error <= error_threshold

def forward_propagation(model, input_data, weights):
    layers = model["layers"]
    output_layer = [input_data]

    for i, layer in enumerate(layers):
        activation_function = layer["activation_function"]
        weight_matrix = weights[i]
        bias = layer.get("bias", 1)

        input_activation = output_layer[-1]
        # print("Dimensi input_activation:", input_activation.shape)
        # print(input_activation)
        if len(input_activation.shape) == 1:
            input_activation_with_bias = np.insert(input_activation, 0, bias)
            input_activation_with_bias = input_activation_with_bias.reshape(1, -1)
        else:
            batch_size = input_activation.shape[0]
            bias_vector = np.ones((batch_size, 1)) * bias
            input_activation_with_bias = np.concatenate((bias_vector, input_activation), axis=1)
        # print("Dimensi input_activation_with_bias:", input_activation_with_bias.shape)
        # print(input_activation_with_bias)
        # print("Dimensi weight_matrix:", weight_matrix.shape)
        # print(weight_matrix)

        output_linear_combination = np.dot(input_activation_with_bias, weight_matrix)

        if activation_function == "linear":
            activation_result = linear(output_linear_combination)
        elif activation_function == "relu":
            activation_result = relu(output_linear_combination)
        elif activation_function == "sigmoid":
            activation_result = sigmoid(output_linear_combination)
        elif activation_function == "softmax":
            activation_result = softmax(output_linear_combination)

        output_layer.append(activation_result)

    return output_layer

def back_propagation(model, input, initial_weights, target, learning_parameter, batch_size, stopped_by, max_iteration, error_threshold):

    i=0
    error = np.inf

    while not terminate_condition(stopped_by, max_iteration, error_threshold, i, error):
        i += 1
        for batch in range(int(input.shape[0]/batch_size)):
            input_batch = input[batch*batch_size:(batch+1)*batch_size]
            target_batch = target[batch*batch_size:(batch+1)*batch_size]
            output_forward = forward_propagation(model, input_batch, initial_weights)
            delta_weight = 0
            for id_data in range(batch_size):
                # Calculate error each output unit
                error_k = []
                for i in range(len(output_forward[-1])):
                    # masih sigmoid
                    activation_function = model["layers"][-1]["activation_function"]
                    output = output_forward[-1][i]
                    target = target_batch[id_data][i]
                    net = np.dot(output_forward[-2], initial_weights[-1][:,i])
                    weights = initial_weights[-1]
                    x = output_forward[-2][i]
                    error = delta_error(activation_function, target, output, net, weights, x, True)
                    error_k.append(error)

                # Calculate error each hidden unit
                error_h = []
                # for each hidden layer
                for j in range(len(output_forward)-2, 0, -1):
                    error_h_j = []
                    for k in range(len(output_forward[j])):
                        # masih sigmoid
                        error_h_j.append(output_forward[j][k]*(1 - output_forward[j][k])*np.dot(error_k, initial_weights[j][:,k]))
                    error_h.append(error_h_j)

                # Update delta weight


            # Update weight per batch

            # Calculate error

        



In [16]:
# Load JSON input
with open("test/backpropagation/softmax_two_layer.json", "r") as file:
    data = json.load(file)

# Extract data
model = data["case"]["model"]
input_data = np.array(data["case"]["input"])
initial_weights = [np.array(layer_weights) for layer_weights in data["case"]["initial_weights"]]
target = np.array(data["case"]["target"])
learning_parameters = data["case"]["learning_parameters"]
learning_rate = data["case"]["learning_parameters"]["learning_rate"]
batch_size = data["case"]["learning_parameters"]["batch_size"]
max_iteration = data["case"]["learning_parameters"]["max_iteration"]
error_threshold = data["case"]["learning_parameters"]["error_threshold"]
stopped_by = data["expect"]["stopped_by"]
final_weights = [np.array(layer_weights) for layer_weights in data["expect"]["final_weights"]]

In [17]:
output_layer = forward_propagation(model, input_data, initial_weights)
print(output_layer)

[array([[ 3.99,  2.96],
       [-0.71,  2.8 ],
       [-2.43, -0.2 ],
       [-1.9 ,  2.62],
       [-2.58,  1.43],
       [-3.43, -0.25],
       [ 1.15, -2.3 ],
       [ 4.28,  3.45]]), array([[0.   , 0.595, 0.   , 0.003],
       [0.451, 0.109, 0.   , 0.   ],
       [0.323, 0.   , 0.363, 0.   ],
       [0.552, 0.   , 0.028, 0.   ],
       [0.501, 0.   , 0.215, 0.   ],
       [0.418, 0.   , 0.468, 0.   ],
       [0.   , 0.   , 0.215, 0.245],
       [0.017, 0.673, 0.   , 0.   ]]), array([[0.58685036, 0.41314964],
       [0.53612693, 0.46387307],
       [0.51726314, 0.48273686],
       [0.52308358, 0.47691642],
       [0.51561492, 0.48438508],
       [0.50626967, 0.49373033],
       [0.54428366, 0.45571634],
       [0.59008578, 0.40991422]])]


In [ ]:
# backward propagation with sklearn
from sklearn.neural_network import MLPClassifier

In [44]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

# Load dataset
dataset = pd.read_csv("test/backpropagation/iris.csv")
learning_data = dataset.drop(columns=["Species"])
target_data = dataset["Species"]

mlp = MLPClassifier(hidden_layer_sizes=(5,9,3), max_iter=1000, activation='logistic', learning_rate='constant', learning_rate_init=0.01,)
mlp.fit(learning_data, target_data)

y_pred = mlp.predict(learning_data)
print("Predicted values:")
print(y_pred)
print("Actual values:")
print(target_data)
print("Accuracy:", mlp.score(learning_data, target_data))

Predicted values:
['Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor

c:\Users\altha\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
